In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import glob
import re
from bs4 import BeautifulSoup
from io import StringIO

In [24]:
"""
df: dataframe
path: str path the store df
"""
def to_csv(df, path):
    return df.to_csv(path)

def to_html(df, path):
    return df.to_html(path)

def to_tsv(df, path):
    return df.to_csv(path, sep='\t')
    
"""
df_type: data frame type to be returned any of (csv, html, tsv)
task: QA task to be performed any of (arithmetic, item)
row_size: row size of the dataset
col_size: col size of the dataset
dir: path to store the dataset 
file_name: name of file
"""
def generate_dataset(df_type, task, row_size, col_size, dir, file_name):
    columns = ['Col ' + str(i+1) for i in range(col_size)]
    rows = ['Row ' + str(i+1) for i in range(row_size)]
    if task == 'arithmetic': 
        df = pd.DataFrame(np.random.randint(1, 11, size=(row_size, col_size)), columns=columns, index=rows)
    elif task == 'item':
        df = pd.DataFrame(np.random.choice(list(string.ascii_uppercase), size=(row_size, col_size)), columns=columns, index=rows)
    df_type_dict = {'csv': to_csv, 'html': to_html, 'tsv': to_tsv}
    
    if not os.path.exists(dir):
        os.makedirs(dir)
    path = os.path.join(dir, file_name + '.' + df_type)
    return df_type_dict[df_type](df, path)


In [25]:
"""
generate nxn dim data
N_list: list(int) of NxN dim data to generate
csv
    Train 14,000 samples
    Test 4300 samples
    Val 4300 samples
    Total 22600 samples
"""
def generate_n_data_set(t, dir):
    # 5 samples
    # Each with 2 tasks (arithmetic, item)
    # Resulting with 10 samples
    N = [4,6,8,10,12]
    j = 0
    for i in range(t): 
        for n in N: #5
            generate_dataset('tsv', 'arithmetic', n, n, dir, f'arithmetic_{j}')
            generate_dataset('csv', 'arithmetic', n, n, dir, f'arithmetic_{j}')
            generate_dataset('html', 'arithmetic', n, n, dir, f'arithmetic_{j}')
        
            generate_dataset('tsv', 'item', n, n, dir, f'item_{j}')
            generate_dataset('csv', 'item', n, n, dir, f'item_{j}')
            generate_dataset('html', 'item', n, n, dir, f'item_{j}')
            j += 1

In [26]:
# generate n x n data
generate_n_data_set(1400, '../datasets/self_generated_v2/csv/0-csv') #train - 1400 * 2 * 5 = 14000
generate_n_data_set(430, '../datasets/self_generated_v2/csv/1-csv') #test - 430 * 2 * 5 = 4300
generate_n_data_set(50, '../datasets/self_generated_v2/csv/2-csv') # val - 50 * 2 * 5 = 500


In [88]:
# Arithmetic operation functions
def get_col_max(df, col_name):
    return df[col_name].max()

def get_col_min(df, col_name):
    return df[col_name].min()

def get_col_sum(df, col_name):
    return df[col_name].sum()

def get_row_max(df, row_num):
    return max(df.iloc[row_num].tolist()[1:])

def get_row_min(df, row_num):
    return min(df.iloc[row_num].values.tolist()[1:])

def get_row_sum(df, row_num):
    return sum(df.iloc[row_num].values.tolist()[1:])

# Item operation functions
def get_col_item(df, col_name):
    return ','.join(df[col_name].tolist())

def get_row_item(df, row_num):
    return ','.join(df.iloc[row_num].values.tolist()[1:])
    
"""
A qa.csv file contain all the questions and their corresponding context and answer
question: str (arithmetic operations include fine the max, min, sum)
answer: str
context: str (only the name of the table)
id: unique str
task: optional enum (“arithmetic” and “item”)
direction: optional enum (“row” or “col”)
size: optional tuple[int] 
"""
ARITHMETIC_OPERATIONS = ['maximum', 'minimum']
def get_question_answer(df, task, direction):
    question = ""
    answer = ""
    if (task == 'arithmetic'):
        # randomly select an operation to perform
        operation_ind = np.random.randint(0,len(ARITHMETIC_OPERATIONS)) # 0, 1, 2
        operation = ARITHMETIC_OPERATIONS[operation_ind]
        col_arithmetic_operation_dict = {'maximum': get_col_max, 'minimum': get_col_min}
        row_arithmetic_operation_dict = {'maximum': get_row_max, 'minimum': get_row_min}
        
        if (direction == 'col'):
            col_num = np.random.randint(0, df.shape[1]-1) # col nums are added one more
            col_name = f"Col {col_num+1}"
            question = f"What is the {operation} of {col_name}?"
            answer = col_arithmetic_operation_dict[operation](df, col_name)
            
        elif (direction == 'row'):
            row_num = np.random.randint(0, df.shape[0])   
            row_name = f"Row {row_num+1}"
            question = f"What is the {operation} of {row_name}?"
            answer = row_arithmetic_operation_dict[operation](df, row_num)

    elif (task == 'item'):
        if (direction == 'col'):
            col_num = np.random.randint(0, df.shape[1]-1) # col nums are added one more
            col_name = f"Col {col_num+1}"
            question = f"Please list all items in {col_name} and separate them by comma."
            answer = get_col_item(df, col_name)
            
        elif (direction == 'row'):
            row_num = np.random.randint(0, df.shape[0])   
            row_name = f"Row {row_num+1}"
            question = f"Please list all items in {row_name} and separate them by comma."
            answer = get_row_item(df, row_num)
          
    return [question, answer]

def get_context(file_path):
    idx = file_path.find('csv')
    return file_path[idx:]

def get_id(prefix, i):
    return prefix + str(i)

def get_task(df):
    series = df.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())[1:] # col nums are added one more
    if (series.all()):
        return 'arithmetic'
    return 'item'

def get_size(df):
    reshaped = (df.shape[0], df.shape[1]-1)# col nums are added one more
    return str(reshaped)

def read_html(path):
    table = BeautifulSoup(open(path,'r').read()).find('table')
    df = pd.read_html(StringIO(str(table)))[0]
    return df

def get_direction():
    DIRECTIONS = ['row', 'col']
    return np.random.choice(DIRECTIONS)

def tables_to_dataset(path, file_prefix, file_name):
    file_paths = glob.glob(path)
    dataset_df = pd.DataFrame(columns=['question', 'answer', 'context', 'id', 'task', 'direction', 'size'])
    i = 0
    for file_path in file_paths: 
        # Read file - only use csv
        df = pd.read_csv(file_path)
        # Define task
        task = get_task(df)
        # Generate q, a for all dims
        direction = get_direction()
        q,a = get_question_answer(df, task, direction)
        dataset_df.loc[i] = [q, a, get_context(file_path), get_id(file_prefix, i), task, direction, get_size(df)]
        i += 1
    dataset_path = '../datasets/self_generated/data/%s.csv'%(file_name)
    return dataset_df.to_csv(dataset_path, index=False)

In [89]:
# generate qa Dataset
train_path = '../datasets/self_generated/csv/0-csv/*.csv'
test_path = '../datasets/self_generated/csv/1-csv/*.csv'
val_path = '../datasets/self_generated/csv/2-csv/*.csv'

tables_to_dataset(train_path, "nt", "train")
tables_to_dataset(test_path, "nu", "test")
tables_to_dataset(val_path, "ns", "val")

In [14]:
"""
Process wtq
"""
def wiki_tables_to_dataset():
    dataset_df = pd.DataFrame(columns=['question', 'answer', 'context', 'id', 'task', 'direction', 'size'])
    dest_to_source_d = {'train.csv': 'training.tsv', 'test.csv' : 'pristine-unseen-tables.tsv', 'val.csv': 'pristine-seen-tables.tsv'}
    for dest, source in dest_to_source_d.items():
        source_file_path = '../datasets/wtq/original_data/%s'%(source)
        df = pd.read_csv(source_file_path, sep='\t', on_bad_lines='skip')
        dataset_df['question'] = df['utterance']
        dataset_df['answer'] = df['targetValue']
        dataset_df['context'] = df['context']
        dataset_df['id'] = df['id']
        
        dest_path = '../datasets/wtq/data/%s'%(dest)
        dataset_df.to_csv(dest_path, index=False)


In [15]:
wiki_tables_to_dataset()